In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# Cargar datos
df = pd.read_csv('data.csv', parse_dates=['fecha'], dayfirst=True)

# Ingeniería de características temporales
df['dia_del_año'] = df['fecha'].dt.dayofyear
df['sin_dia'] = np.sin(2 * np.pi * df['dia_del_año'] / 365.25)
df['cos_dia'] = np.cos(2 * np.pi * df['dia_del_año'] / 365.25)

# Transformar a problema de clasificación
df['lluvia'] = (df['precipitacion_total'] > 0).astype(int)

# Preparar datos
X = df[['temp_max', 'temp_min', 'presion_max', 'presion_min', 
        'rocio_max', 'rocio_min', 'viento_promedio', 'radiacion', 
        'sin_dia', 'cos_dia']]
y = df['lluvia']

# Escalado y división
scaler = StandardScaler()
X_escalado = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_escalado, y, test_size=0.3, random_state=42)

# Definir parámetros para GridSearchCV
param_grids = {
    'SVM': {
        'model': SVC(probability=True, random_state=42),
        'params': {
            'C': [0.1, 1, 10],
            'kernel': ['rbf', 'linear'],
            'gamma': ['scale', 'auto']
        }
    },
    'Decision Tree': {
        'model': DecisionTreeClassifier(random_state=42),
        'params': {
            'max_depth': [3, 5, 7, 10, None],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4]
        }
    },
    'Random Forest': {
        'model': RandomForestClassifier(random_state=42),
        'params': {
            'n_estimators': [50, 100, 200],
            'max_depth': [3, 5, 7, None],
            'min_samples_split': [2, 5]
        }
    },
    'XGBoost': {
        'model': XGBClassifier(random_state=42, eval_metric='logloss'),
        'params': {
            'n_estimators': [50, 100, 200],
            'max_depth': [3, 5, 7],
            'learning_rate': [0.01, 0.1, 0.2]
        }
    },
    'KNN': {
        'model': KNeighborsClassifier(),
        'params': {
            'n_neighbors': [3, 5, 7, 9],
            'weights': ['uniform', 'distance'],
            'metric': ['euclidean', 'manhattan']
        }
    }
}

# Naive Bayes no usa GridSearchCV ya que tiene pocos parámetros
naive_bayes = GaussianNB()

# Entrenar y evaluar modelos
resultados = {}

# Modelos con GridSearchCV
for nombre, config in param_grids.items():
    print(f"\nOptimizando hiperparámetros para {nombre}...")
    
    # GridSearchCV
    grid_search = GridSearchCV(
        estimator=config['model'],
        param_grid=config['params'],
        cv=5,
        scoring='accuracy',
        n_jobs=-1,
        verbose=1
    )
    
    # Entrenar con GridSearch
    grid_search.fit(X_train, y_train)
    
    # Mejor modelo
    best_model = grid_search.best_estimator_
    
    # Predicciones
    y_pred = best_model.predict(X_test)
    y_pred_proba = best_model.predict_proba(X_test)[:, 1] if hasattr(best_model, "predict_proba") else None
    
    # Métricas de evaluación
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    # Guardar resultados
    resultados[nombre] = {
        'modelo': best_model,
        'predicciones': y_pred,
        'probabilidades': y_pred_proba,
        'mejores_parametros': grid_search.best_params_,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }
    
    print(f" {nombre} optimizado - Mejores parámetros: {grid_search.best_params_}")
    print(f"   Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")

# Entrenar Naive Bayes (sin GridSearchCV)
print(f"\n🏃 Entrenando Naive Bayes...")
naive_bayes.fit(X_train, y_train)
y_pred_nb = naive_bayes.predict(X_test)
y_pred_proba_nb = naive_bayes.predict_proba(X_test)[:, 1]

# Métricas para Naive Bayes
accuracy_nb = accuracy_score(y_test, y_pred_nb)
precision_nb = precision_score(y_test, y_pred_nb)
recall_nb = recall_score(y_test, y_pred_nb)
f1_nb = f1_score(y_test, y_pred_nb)

resultados['Naive Bayes'] = {
    'modelo': naive_bayes,
    'predicciones': y_pred_nb,
    'probabilidades': y_pred_proba_nb,
    'mejores_parametros': 'N/A',
    'accuracy': accuracy_nb,
    'precision': precision_nb,
    'recall': recall_nb,
    'f1': f1_nb
}

print(f"Naive Bayes entrenado")
print(f"   Accuracy: {accuracy_nb:.4f}, Precision: {precision_nb:.4f}, Recall: {recall_nb:.4f}, F1: {f1_nb:.4f}")

# Comparación final de modelos
print("\n" + "="*80)
print("COMPARACIÓN FINAL DE MODELOS")
print("="*80)

# Crear DataFrame para comparación
comparacion = []
for nombre, metrics in resultados.items():
    comparacion.append({
        'Modelo': nombre,
        'Accuracy': f"{metrics['accuracy']:.4f}",
        'Precision': f"{metrics['precision']:.4f}",
        'Recall': f"{metrics['recall']:.4f}",
        'F1-Score': f"{metrics['f1']:.4f}",
        'Mejores Parámetros': str(metrics['mejores_parametros'])[:50] + "..." if len(str(metrics['mejores_parametros'])) > 50 else metrics['mejores_parametros']
    })

df_comparacion = pd.DataFrame(comparacion)
print(df_comparacion.to_string(index=False))

# Mejor modelo basado en F1-Score (balance entre precision y recall)
mejor_modelo_nombre = max(resultados.items(), key=lambda x: x[1]['f1'])[0]
mejor_modelo = resultados[mejor_modelo_nombre]['modelo']

print(f"\n MEJOR MODELO: {mejor_modelo_nombre} (F1-Score: {resultados[mejor_modelo_nombre]['f1']:.4f})")

# Reporte de clasificación detallado del mejor modelo
print(f"\nREPORTE DE CLASIFICACIÓN - {mejor_modelo_nombre}:")
print(classification_report(y_test, resultados[mejor_modelo_nombre]['predicciones']))